In [1]:
import numpy as np
import pyvista as pv
import torch
import matplotlib.pyplot as plt

In [2]:
# vertical mask
film = np.zeros((20, 100, 160, 3))

bottom = 100
height = 104

density = 10

radius = 30

center = 50

film[:, :45, bottom:height, 2] = density
film[:, 55:, bottom:height, 2] = density
# film[:, :, 0:bottom, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
# film[:, :, 0:bottom, 1] = 0 # bottom
# film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
depo1 = torch.Tensor(film[:, :100, :, 1]>0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :100, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(film[:, :100, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [3]:
etchfilm = film

In [ ]:
etchfilm = np.load('./bosch_data_1017_ratio08_hole/bosch_sf_step_1_c4f8.npy')

In [4]:
import Multi_Species_bosch_etching_countByReaction_mirror_func

In [5]:
logname = 'Multi_species_benchmark_1022_hole_ratio01'

testEtch = Multi_Species_bosch_etching_countByReaction_mirror_func.etching(inputMethod='bunch', depo_or_etching='etching', 
                    etchingPoint = np.array([center, center, bottom-30]),depoPoint = np.array([center, center, bottom-30]),
                    density=density, center_with_direction=np.array([[int(etchfilm.shape[0]/2),int(etchfilm.shape[1]/2),150]]), 
                    range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]]), InOrOut=[1], yield_hist=np.array([None]),
                    maskTop=40, maskBottom=98, maskStep=10, maskCenter=[int(etchfilm.shape[0]/2), int(etchfilm.shape[1]/2)],backup=False, 
                    mirrorGap=5,
                    reaction_type=False, param = [1.6, -0.7],n=1,
                    celllength=1e-5, kdtreeN=5, filmKDTree=np.array([[2, 0], [3, 1]]),weight=-0.2, tstep=1e-5,
                    substrateTop=bottom,posGeneratorType='top', logname=logname)

In [6]:
react_table = np.array([[[0.1, -1, 0, 0], [1.0, 0,  0, 0], [1.0, 0, 0, 0]],
                        [[0.8, -1, 1, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[1.0,  0, 0, 0], [1.0, 0, -2, 0], [1.0, 0, 0, 0]]])

In [7]:
Multi_Species_bosch_etching_countByReaction_mirror_func.react_table = react_table

In [8]:
Multi_Species_bosch_etching_countByReaction_mirror_func.react_table

array([[[ 0.1, -1. ,  0. ,  0. ],
        [ 1. ,  0. ,  0. ,  0. ],
        [ 1. ,  0. ,  0. ,  0. ]],

       [[ 0.8, -1. ,  1. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ]],

       [[ 1. ,  0. ,  0. ,  0. ],
        [ 1. ,  0. , -2. ,  0. ],
        [ 1. ,  0. ,  0. ,  0. ]]])

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(4e4), int(4e6), int(1e5),2, 10, 100)
# np.save('./bosch_data_1015_test/bosch_sf_step_sf01', etchfilm)

(30, 110, 160)


particle input:  93%|██████████████████████████████████████████▊   | 93.0/100 [34:31<02:35, 22.3s/B]


KeyboardInterrupt: 

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(6e4), int(1e7), int(4e5),2, 10, 100)

In [ ]:
celllength=1e-5
parcelAr = np.array([[etchfilm.shape[0]*celllength, etchfilm.shape[0]*celllength, 199*celllength, 0, 0, 1, etchfilm.shape[0], etchfilm.shape[0], 199, 2]])
step3 = testEtch.inputParticle(etchfilm, parcelAr, 'updown', 2, int(5e4), int(12e7), int(1e5), 4, 10, i)

In [ ]:
np.save('./benchmark_etching/trench_etch_rate_09_12e5', etchfilm)

In [ ]:
def attachPolymer(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] == 0 

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & neighbors

    # 将孤立的单元格设为0
    film[condition, 1] = 10

    all_film = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,0] == 10)
    film[all_film, 1] = 0
    all_film2 = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,2] == 10)
    film[all_film2, 1] = 0
    return film

In [ ]:
def thickPolymer(film):  # fast scanZ
    filmC = film[:,:,:,1]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,0] == 10 

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & neighbors

    # 将孤立的单元格设为0
    film[condition, 1] = 10

    all_film = np.logical_and(film[:,:,:,1] == 10, film[:,:,:,0] == 10)
    film[all_film, 0] = 0
    return film

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[etchfilm.shape[0]*celllength, etchfilm.shape[0]*celllength, 199*celllength, 0, 0, 1, etchfilm.shape[0], etchfilm.shape[0], 199, 0]])

for i in range(cicle):
    step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(5e4), int(12e7), int(1.2e6),2, 10, i)
    indice_step1 = etchfilm[:, :, :, 0] >= 9
    etchfilm[indice_step1, 0] = 10
    depth = step1[1] 
    parcel = step1[2]
    if depth < 50:
        film = np.zeros((etchfilm.shape[0], etchfilm.shape[1], etchfilm.shape[2] + 50, 3))
        film[:, :, 50:] = etchfilm
        film[:, :, :50, 0] = 10
        etchfilm = film
        parcel[:, 2] += 50*celllength
        parcel[:, 8] += 50
    print('film:', etchfilm.shape)
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_sf'.format(i), etchfilm)
    etchfilm = attachPolymer(etchfilm)
    # etchfilm = thickPolymer(etchfilm)
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    parcelAr = np.array([[etchfilm.shape[0]*celllength, etchfilm.shape[0]*celllength, 199*celllength, 0, 0, 1, etchfilm.shape[0], etchfilm.shape[0], 199, 0]])
    step3 = testEtch.inputParticle(etchfilm, parcelAr, 'updown', 2, int(5e4), int(12e7), int(1e5), 4, 10, i)
    indice_c3 = np.logical_and(etchfilm[:, :, :, 1]>9, etchfilm[:, :, :, 1]<10)
    etchfilm[indice_c3, 1] = 10
    indice_step2 = etchfilm[:, :, :, 0] != 10
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[101*celllength, 101*celllength, 199*celllength, 0, 0, 1, 101, 101, 199, 0]])

for i in range(cicle):
    step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(3e4), int(8e6), int(4e5),2, 10, i)
    indice_step1 = etchfilm[:, :, :, 0] >= 9
    etchfilm[indice_step1, 0] = 10
    depth = step1[1] 
    parcel = np.array([[101*celllength, 101*celllength, 199*celllength, 0, 0, 1, 101, 101, 199, 0]])
    if depth < 50:
        film = np.zeros((etchfilm.shape[0], etchfilm.shape[1], etchfilm.shape[2] + 50, 3))
        film[:, :, 50:] = etchfilm
        film[:, :, :50, 0] = 10
        etchfilm = film
        parcel[:, 2] += 50*celllength
        parcel[:, 8] += 50
    print('film:', etchfilm.shape)
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_sf'.format(i), etchfilm)
    etchfilm = attachPolymer(etchfilm)
    # etchfilm = thickPolymer(etchfilm)
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    step3 = testEtch.inputParticle(etchfilm, parcel, 'updown', 2, int(3e4), int(8e6), int(6e4), 4, 10, i)
    parcel = np.array([[101*celllength, 101*celllength, 199*celllength, 0, 0, 1, 101, 101, 199, 0]])
    indice_c3 = np.logical_and(etchfilm[:, :, :, 1]>9, etchfilm[:, :, :, 1]<10)
    etchfilm[indice_c3, 1] = 10
    indice_step2 = etchfilm[:, :, :, 0] != 10
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1021_ratio01_hole/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])

for i in range(cicle):
    step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(2e4),int(4e5),2, 10, i)
    indice_step1 = etchfilm[:, :, :, 0] >= 9
    etchfilm[indice_step1, 0] = 10
    depth = step1[1] 
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    if depth < 50:
        film = np.zeros((100, 100, etchfilm.shape[2] + 50, 3))
        film[:, :, 50:] = etchfilm
        film[:, :, :50, 0] = 10
        etchfilm = film
        parcel[:, 2] += 50*celllength
        parcel[:, 8] += 50
    print('film:', etchfilm.shape)
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_sf'.format(i), etchfilm)
    step2 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 1, int(2e4),int(8e4), 1, 10, i)
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    indice_step2 = etchfilm[:, :, :, 0] < 0
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    step3 = testEtch.inputParticle(etchfilm, parcel, 'updown', 2, int(2e4),int(6e4), 4, 10, i)
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    indice_c3 = np.logical_and(etchfilm[:, :, :, 1]>9, etchfilm[:, :, :, 1]<10)
    etchfilm[indice_c3, 1] = 10
    indice_step2 = etchfilm[:, :, :, 0] != 10
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [ ]:
np.save('./bosch_data_1001_ratio08/bosch_sf_stepTest_c4f8', etchfilm)

In [ ]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 10

indice_step2 = etching[:, :, :, 0] != 10
etching[indice_step2, 0] = 0
indice_c2 = etching[:, :, :, 1] != 0
etching[indice_c2, 1] = 10

In [ ]:
etching = np.load('./bosch_data_1015_test/bosch_sf_step_sf.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:, :50, :,1]==10, etching[:, :50, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :50, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :50, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :50, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [10]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:, :, :,1]==10, etchfilm[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52533/index.html?ui=P_0x225544c05c0_0&reconnect=auto" class="pyvis…

In [ ]:
etchfilm = np.load('./bosch_data_1017_ratio08_hole/bosch_sf_step_13_sf.npy')

In [ ]:
etchfilm = np.load('./bosch_data_1018_speedTest/bosch_sf_step_12_sf_nolimit.npy')

In [ ]:
etchfilm = np.load('./bosch_data_1018_speedTest/bosch_sf_step_12_sf_nolimit_rate01.npy')

In [ ]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:50, :, :,1]==10, etchfilm[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:50, :, :,1]==10, etchfilm[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:50, :, :,1]==10, etchfilm[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()